# x. Model Inference

>Model yang sudah dilatih akan dicoba pada data yang bukan termasuk ke dalam train-set ataupun test-set. Data ini harus dalam format yang asli, bukan data yang sudah di-scaled.

Pada bagian ini akan dilakukan proses model inference dengan menggunakan data `inference set` dan melakukan preprocessing yang sama seperti yang dilakukan sebelumnya **(tanpa proses handling outlier dan handling missing value)** serta melakukan prediksi terhadap data `inference-set`.

## Import Libraries

In [1]:
## REQUIRED LIBRARIES

# For load the files of model
import pickle
import json
import joblib

# For data wrangling
import numpy as np  # linear algebra
import pandas as pd  # data processing

## Load model

In [2]:
# Load All Files of XGB after hyperparameter tuning

with open('xgb_gridcv_best.pkl', 'rb') as file_1:
  xgb_gridcv_best = pickle.load(file_1)

with open('model_scaler1.pkl', 'rb') as file_2:
  model_scaler1 = pickle.load(file_2)

with open('model_scaler2.pkl', 'rb') as file_3:
  model_scaler2 = pickle.load(file_3)

with open('list_num_cols.txt', 'r') as file_4:
  list_num_cols = json.load(file_4)

with open('list_cat_cols.txt', 'r') as file_5:
  list_cat_cols = json.load(file_5)

## Load data

In [3]:
# load data
df_inf = pd.read_csv('/content/data_inference.csv')
df_inf

,limit_balance,gender,education_level,marital_status,age,pay_1,pay_2,pay_3,pay_4,pay_5,...,bill_amt_4,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,default_payment
0,240000,2,1,2,25,1,-1,-1,-1,-1,...,2417,0,0,587,0,2417,0,0,7642,0
1,170000,2,1,2,42,-1,-1,-1,-1,-1,...,316,948,632,316,316,316,948,0,0,0
2,30000,1,3,2,31,0,0,0,0,0,...,29784,0,0,1703,1200,2196,2500,0,0,1
3,500000,1,1,1,46,-2,-2,-2,-2,-2,...,1400,0,0,0,1656,1400,0,0,0,0
4,100000,1,2,1,46,2,2,2,0,0,...,18036,18446,18840,1300,0,700,700,700,800,1
5,230000,2,2,1,32,0,0,0,0,0,...,34158,22872,22592,2500,3000,3000,1500,1500,2000,1
6,260000,1,1,1,43,1,-1,2,0,-1,...,3804,5009,3035,5182,9,2998,5034,3050,2965,0
7,390000,2,1,2,30,0,0,0,0,0,...,57852,55903,53516,5000,2506,3000,2500,2000,2000,0
8,230000,2,3,2,47,0,0,0,0,0,...,94225,96616,95849,10000,5000,4000,4000,5000,4000,0
9,30000,2,1,2,27,0,0,0,0,0,...,22544,24061,23428,2000,3000,2000,2000,1000,0,1


## Memisahkan antara kolom Numerikal dan kolom Kategorikal

In [4]:
# Split between Numerical Columns and Categorical Columns

df_inf_num = df_inf[list_num_cols]
df_inf_cat = df_inf[list_cat_cols]

In [5]:
# display num_cols
df_inf_num

,limit_balance,age,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6
0,240000,25,0,587,0,2417,0,0,587,0,2417,0,0,7642
1,170000,42,3480,316,316,316,948,632,316,316,316,948,0,0
2,30000,31,27838,28791,27788,29784,0,0,1703,1200,2196,2500,0,0
3,500000,46,0,0,1656,1400,0,0,0,1656,1400,0,0,0
4,100000,46,17497,18219,17629,18036,18446,18840,1300,0,700,700,700,800
5,230000,32,57259,46226,36089,34158,22872,22592,2500,3000,3000,1500,1500,2000
6,260000,43,-10,5172,2800,3804,5009,3035,5182,9,2998,5034,3050,2965
7,390000,30,60819,61300,59328,57852,55903,53516,5000,2506,3000,2500,2000,2000
8,230000,47,147300,121855,102074,94225,96616,95849,10000,5000,4000,4000,5000,4000
9,30000,27,20234,20476,21991,22544,24061,23428,2000,3000,2000,2000,1000,0


In [6]:
# display cat_cols
df_inf_cat

,gender,education_level,marital_status,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6
0,2,1,2,1,-1,-1,-1,-1,-2
1,2,1,2,-1,-1,-1,-1,-1,0
2,1,3,2,0,0,0,0,0,-2
3,1,1,1,-2,-2,-2,-2,-2,-2
4,1,2,1,2,2,2,0,0,0
5,2,2,1,0,0,0,0,0,0
6,1,1,1,1,-1,2,0,-1,-1
7,2,1,2,0,0,0,0,0,0
8,2,3,2,0,0,0,0,0,0
9,2,1,2,0,0,0,0,0,2


## Feature Scaling

In [8]:
# create dataFrame df_inf_num_1 and df_inf_num_2
df_inf_num_1 = pd.DataFrame(df_inf_num['age'])
df_inf_num_2 = pd.DataFrame(df_inf_num[['limit_balance', 'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6', 'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6']])

# Feature Scaling
df_inf_num_scaled1 = model_scaler1.transform(df_inf_num_1)
df_inf_num_scaled2 = model_scaler2.transform(df_inf_num_2)
df_inf_num_scaled1

array([[0.08333333],
       [0.4375    ],
       [0.20833333],
       [0.52083333],
       [0.52083333],
       [0.22916667],
       [0.45833333],
       [0.1875    ],
       [0.54166667],
       [0.125     ]])

## Menggabungkan (concate) kolom Numerikal dan kolom Kategorikal

In [9]:
# concate numerical columns and categorical columns
df_inf_final_1 = np.concatenate([df_inf_num_scaled1, df_inf_num_scaled2, df_inf_cat], axis=1)
df_inf_final_1

array([[ 0.08333333,  0.55555556, -0.37313038, -0.34376521, -0.36326754,
        -0.31691412, -0.36505668, -0.36354155, -0.3969428 , -0.52070419,
         0.10762107, -0.43081081, -0.45432031,  1.57631098,  2.        ,
         1.        ,  2.        ,  1.        , -1.        , -1.        ,
        -1.        , -1.        , -2.        ],
       [ 0.4375    ,  0.16666667, -0.32052784, -0.34795238, -0.3579479 ,
        -0.35502508, -0.34649647, -0.35110538, -0.4637574 , -0.44235061,
        -0.4182205 , -0.17459459, -0.45432031, -0.4174276 ,  2.        ,
         1.        ,  2.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        ,  0.        ],
       [ 0.20833333, -0.61111111,  0.04765972,  0.09200961,  0.10452422,
         0.17950788, -0.36505668, -0.36354155, -0.12179487, -0.22315894,
         0.05230885,  0.24486486, -0.45432031, -0.4174276 ,  1.        ,
         3.        ,  2.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.    

In [10]:
# Create DataFrame of `df_inf_final_1`

df_inf_final_1 = pd.DataFrame(df_inf_final_1, columns=['age', 'limit_balance', 'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6', 'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6', 'gender', 'education_level', 'marital_status', 'pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6'])
df_inf_final_1

,age,limit_balance,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,...,pay_amt_6,gender,education_level,marital_status,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6
0,0.083333,0.555556,-0.373130,-0.343765,-0.363268,-0.316914,-0.365057,-0.363542,-0.396943,-0.520704,...,1.576311,2.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-2.0
1,0.437500,0.166667,-0.320528,-0.347952,-0.357948,-0.355025,-0.346496,-0.351105,-0.463757,-0.442351,...,-0.417428,2.0,1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
2,0.208333,-0.611111,0.047660,0.092010,0.104524,0.179508,-0.365057,-0.363542,-0.121795,-0.223159,...,-0.417428,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,-2.0
3,0.520833,2.000000,-0.373130,-0.352835,-0.335390,-0.335362,-0.365057,-0.363542,-0.541667,-0.110092,...,-0.417428,1.0,1.0,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
4,0.520833,-0.222222,-0.108651,-0.071336,-0.066496,-0.033594,-0.003916,0.007182,-0.221154,-0.520704,...,-0.208714,1.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0
5,0.229167,0.500000,0.492378,0.361395,0.244266,0.258850,0.082738,0.081012,0.074704,0.223159,...,0.104357,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.458333,0.666667,-0.373282,-0.272923,-0.316131,-0.291755,-0.266989,-0.303820,0.735947,-0.518473,...,0.356118,1.0,1.0,1.0,1.0,-1.0,2.0,0.0,-1.0,-1.0
7,0.187500,1.388889,0.546190,0.594300,0.635478,0.688646,0.729428,0.689519,0.691075,0.100669,...,0.104357,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.541667,0.500000,1.853408,1.529924,1.355078,1.348431,1.526519,1.522526,1.923817,0.719068,...,0.626141,2.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.125000,-0.611111,-0.067280,-0.036464,0.006936,0.048178,0.106016,0.097463,-0.048570,0.223159,...,-0.417428,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0


In [12]:
# create a new variable containing columns are not used in model
col_not_used = df_inf_final_1[['gender', 'education_level', 'marital_status', 'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6']]

# drop columns in df_inf_final_1 which are not used in model 
df_inf_final_1.drop(col_not_used, axis=1, inplace=True)
df_inf_final_1

,age,limit_balance,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6
0,0.083333,0.555556,-0.396943,-0.520704,0.107621,-0.430811,-0.454320,1.576311,1.0,-1.0,-1.0,-1.0,-1.0,-2.0
1,0.437500,0.166667,-0.463757,-0.442351,-0.418220,-0.174595,-0.454320,-0.417428,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
2,0.208333,-0.611111,-0.121795,-0.223159,0.052309,0.244865,-0.454320,-0.417428,0.0,0.0,0.0,0.0,0.0,-2.0
3,0.520833,2.000000,-0.541667,-0.110092,-0.146915,-0.430811,-0.454320,-0.417428,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
4,0.520833,-0.222222,-0.221154,-0.520704,-0.322112,-0.241622,-0.261695,-0.208714,2.0,2.0,2.0,0.0,0.0,0.0
5,0.229167,0.500000,0.074704,0.223159,0.253535,-0.025405,-0.041552,0.104357,0.0,0.0,0.0,0.0,0.0,0.0
6,0.458333,0.666667,0.735947,-0.518473,0.253035,0.929730,0.384975,0.356118,1.0,-1.0,2.0,0.0,-1.0,-1.0
7,0.187500,1.388889,0.691075,0.100669,0.253535,0.244865,0.096037,0.104357,0.0,0.0,0.0,0.0,0.0,0.0
8,0.541667,0.500000,1.923817,0.719068,0.503817,0.650270,0.921574,0.626141,0.0,0.0,0.0,0.0,0.0,0.0
9,0.125000,-0.611111,-0.048570,0.223159,0.003254,0.109730,-0.179141,-0.417428,0.0,0.0,0.0,0.0,0.0,2.0


## Prediksi menggunakan XGB after hyperparameter tuning

In [13]:
# Predict using XGB

y_pred_inf = xgb_gridcv_best.predict(df_inf_final_1)
y_pred_inf = pd.DataFrame(y_pred_inf, columns= ['Default Payment after tunning'])
y_pred_inf

,Default Payment after tunning
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,0
9,0


In [14]:
# concate `default_payment` in data `df_inf` and the result of price prediction
df_inf_final = pd.concat([df_inf['default_payment'], y_pred_inf], axis=1)
df_inf_final

,default_payment,Default Payment after tunning
0,0,0
1,0,0
2,1,0
3,0,0
4,1,1
5,1,0
6,0,0
7,0,0
8,0,0
9,1,0


Berdasarkan dataFrame di atas, kita dapat mengetahui bagaimana keakuratan setiap model memprediksi apakah customer akan default pada kartu kreditnya. Kita dapat melihat bahwa `ada 3 perbedaan` antara nilai aktual `default_payment` dengan `prediksi default_payment` menggunakan model XGB (dengan hyperparameter tuning). Interpretasinya yaitu model salah memprediksi bahwa seseorang `tidak menunggak pembayaran kredit`, padahal aktualnya orang ini `menunggak dan kartu kreditnya di blokir`.<br>
Pada proses evaluasi model sebelumnya, kita mendapatkan bahwa model yang memiliki performansi terbaik adalah `model XGB yang sudah dilakukan hyperparameter tuning`. Namun ternyata hasil evaluasi model diatas dapat dibuktikan dengan dataFrame di atas, yaitu dari ke-10 data inference di atas, model ini ada melakukan kesalahan prediksi sebanyak `3 data` artinya `hanya 7 data dari 10 data yang ada` yang diprediksi benar oleh `model XGB yang sudah dilakukan hyperparameter tuning`. Sehingga, masih terus dibutuhkan *model improvement* untuk meningkatkan performa dari model ini.
